In [2]:
import pandas as pd


In [3]:
df = pd.read_csv("registros_rio_6746.csv")
#modificando las columns del df:
df["date"] = pd.to_datetime(df["date"])

df = df.sort_values(by="date")
df = df.drop(columns=['es_null', 'year'])

df['altura_7_dias'] = df['altura_value'].shift(periods=-7)
df = df.iloc[:1001].copy() #elimino las ultimas 7 ya que tienen datos del futuro
df

,date,rio_id,lat,lon,altura_value,precipitaciones_value,altura_7_dias
0,2023-01-02,6746-2123,-36.397778,-67.140278,-0.06,0.0,-0.06
1,2023-01-03,6746-2123,-36.397778,-67.140278,-0.06,0.0,-0.06
2,2023-01-04,6746-2123,-36.397778,-67.140278,-0.06,0.0,-0.06
3,2023-01-05,6746-2123,-36.397778,-67.140278,-0.06,0.1,-0.06
4,2023-01-06,6746-2123,-36.397778,-67.140278,-0.06,0.1,-0.06
...,...,...,...,...,...,...,...
996,2025-09-24,6746-2123,-36.397778,-67.140278,0.84,0.0,0.75
997,2025-09-25,6746-2123,-36.397778,-67.140278,0.81,0.0,0.78
998,2025-09-26,6746-2123,-36.397778,-67.140278,0.77,4.9,0.81
999,2025-09-27,6746-2123,-36.397778,-67.140278,0.77,0.0,0.83


In [8]:
print(df.iloc[800])
df.info()

date                     2025-03-12 00:00:00
rio_id                             6746-2123
lat                               -36.397778
lon                               -67.140278
altura_value                           -0.12
precipitaciones_value                    0.0
altura_7_dias                           0.24
Name: 800, dtype: object
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   date                   1001 non-null   datetime64[ns]
 1   rio_id                 1001 non-null   object        
 2   lat                    1001 non-null   float64       
 3   lon                    1001 non-null   float64       
 4   altura_value           1001 non-null   float64       
 5   precipitaciones_value  1001 non-null   float64       
 6   altura_7_dias          1001 non-null   float64       
dtypes: datetime64[ns](

In [ ]:
import numpy as np
#Dividiendo en train y test
set_entrenamiento = df[df['date'] < "2025-03-12"]
set_validacion = df[df['date'] >= "2025-03-12"]

print(set_entrenamiento.min().date)
print(set_entrenamiento.max().date)
print(set_validacion.min().date)
print(set_validacion.max().date)

2023-01-02 00:00:00
2025-03-11 00:00:00
2025-03-12 00:00:00
2025-09-28 00:00:00


In [19]:
from sklearn.preprocessing import MinMaxScaler 

sc = MinMaxScaler(feature_range= (0,1))
set_entrenamiento_escalado = sc.fit_transform(set_entrenamiento)

TypeError: float() argument must be a string or a real number, not 'Timestamp'